# segment 10 seconds for calculating ViSQOLAudio

In [2]:
import random
import wave
from pathlib import Path
import soundfile as sf
from tqdm.notebook import tqdm
import numpy as np

sr = 24_000
segment_size = sr * 10  # 10 sec
base_dir = "/home/shahn/Datasets/jamendo/raw_30s_24khz/99"
to_dir = "/home/shahn/Datasets/jamendo/10s_segment_24khz/99"
for file in Path(base_dir).rglob('*.wav'):
    wave_read = wave.open(str(file))
    assert wave_read.getframerate() == sr
    start_idx = random.randint(0, wave_read.getnframes() - segment_size)
    wave_read.setpos(start_idx)
    segment = wave_read.readframes(segment_size)
    wav = np.frombuffer(segment, dtype=np.int16, count=len(segment)//2, offset=0)
    save_path = to_dir / file.relative_to(base_dir)
    save_path.parent.mkdir(parents=True, exist_ok=True)
    sf.write(save_path, wav, sr, subtype='PCM_16')

# generate a filelist

In [3]:
from tqdm.notebook import tqdm
from pathlib import Path
import re

base_dir = "/home/shahn/Datasets"
directories = [
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p225",  # clean
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p226",  # clean
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p227",  # clean
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p228",  # clean
    "/home/shahn/Datasets/DNS-Challenge4/datasets_24khz/dev_testset", # noisy
    "/home/shahn/Datasets/jamendo/10s_segment_24khz/99"               # music
]
# directories = [
#     "/home/shahn/Datasets/VCTK-0.92/wav16_silence_trimmed/p225",  # clean
#     "/home/shahn/Datasets/VCTK-0.92/wav16_silence_trimmed/p226",  # clean
#     "/home/shahn/Datasets/VCTK-0.92/wav16_silence_trimmed/p227",  # clean
#     "/home/shahn/Datasets/VCTK-0.92/wav16_silence_trimmed/p228",  # clean
#     "/home/shahn/Datasets/DNS-Challenge3/datasets/wideband/dev_testset_wideband", # noisy
#     "/home/shahn/Datasets/jamendo/10s_segment_16khz/99"               # music
# ]
directories_to_exclude = [
]


def is_directory_to_exclude(file: Path, directories_to_exclude) -> bool:
    parents = file.parents
    for dte in directories_to_exclude:
        dte = Path(dte)
        if dte in parents:
            return True
    return False


with open("../../filelists/DNS/DNS_VCTK_jamendo_valid_24khz.txt", "w") as f:
    for directory in directories:
        for file in Path(directory).rglob('*.wav'):
            if is_directory_to_exclude(file, directories_to_exclude):
                continue
            
            m = re.match(r".+_([0-9]+)_mic[12]\.wav", str(file))
            if m and int(m[1]) > 80:
                continue
            
            file = str(file.relative_to(base_dir))
            f.write(f"{file}\n")

In [5]:
from tqdm.notebook import tqdm
from pathlib import Path
import re
import wave

base_dir = "/home/shahn/Datasets"
directories = [
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p225",  # clean
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p226",  # clean
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p227",  # clean
    "/home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p228",  # clean
    "/home/shahn/Datasets/DNS-Challenge4/datasets_24khz/dev_testset", # noisy
    "/home/shahn/Datasets/jamendo/10s_segment_24khz/99"               # music
]
minimum_length = 6
nums = [25, 25, 25, 25, 100, 100]

with open("../../filelists/DNS/DNS_VCTK_jamendo_pesq_24khz.txt", "w") as f:
    for directory, num in zip(directories, nums):
        count = 0
        for file in Path(directory).rglob('*.wav'):
            wave_read = wave.open(str(file))
            sr = wave_read.getframerate()
            assert sr == 24_000
            wav_len = wave_read.getnframes()
            if wav_len >= minimum_length * sr:
                count += 1
                file = str(file.relative_to(base_dir))
                f.write(f"{file}\n")
            if count >= num:
                break
        print(f"{count}: {directory}")

25: /home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p225
25: /home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p226
25: /home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p227
25: /home/shahn/Datasets/VCTK-0.92/wav24_silence_trimmed/p228
100: /home/shahn/Datasets/DNS-Challenge4/datasets_24khz/dev_testset
100: /home/shahn/Datasets/jamendo/10s_segment_24khz/99
